In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
import torch.nn.functional as F

from torch.utils.data import Dataset
torch.manual_seed(0)
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')


In [2]:
# Define your custom dataset
class MyDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# Create dataset
train_dataset = MyDataset(X_train, y_train)
# Create DataLoaders for training and testing
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
del X_train

In [3]:
# Simpler model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(65, 2, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128*240, 65)  
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer
        self.fc3 = nn.Linear(65, 2)
        

    def forward(self, x):
        x = x.view(-1, 65, 256, 240)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = x.view(x.size(0), -1)  # Flatten layer
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
# Initialize the network and print its architecture
model = Net()
print(model)

Net(
  (conv1): Conv2d(65, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=30720, out_features=65, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=65, out_features=2, bias=True)
)


In [4]:
# Use CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight = torch.tensor([0.5,1.0]).to(device))
optimizer = optim.Adam(model.parameters(), lr=1e-4)


# Initialize empty lists to store losses
train_losses = []

# Training loop
for epoch in range(15):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.float()
        labels = labels.type(torch.LongTensor)   # casting to long
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        l1_lambda = 0.0005
        l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss = criterion(outputs, labels)
        loss+=l1_lambda*l1_norm
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Compute training loss
        train_losses.append(loss.item())
    print(f'Epoch {epoch+1}, training loss: {running_loss/len(train_dataloader):.4f}')

# Now without regularization
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# Training loop
for epoch in range(20):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.float()
        labels = labels.type(torch.LongTensor)   # casting to long
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Compute training loss
        train_losses.append(loss.item())
    print(f'Epoch {epoch+1}, training loss: {running_loss/len(train_dataloader):.4f}')

print('Finished Training')

Epoch 1, training loss: 3.0144
Epoch 2, training loss: 2.0268
Epoch 3, training loss: 1.3780
Epoch 4, training loss: 1.0252
Epoch 5, training loss: 0.8956
Epoch 6, training loss: 0.8356
Epoch 7, training loss: 0.8230
Epoch 8, training loss: 0.8027
Epoch 9, training loss: 0.8040
Epoch 10, training loss: 0.8018
Epoch 11, training loss: 0.7677
Epoch 12, training loss: 0.7583
Epoch 13, training loss: 0.7618
Epoch 14, training loss: 0.7770
Epoch 15, training loss: 0.7990
Epoch 1, training loss: 0.7308
Epoch 2, training loss: 0.6901
Epoch 3, training loss: 0.6770
Epoch 4, training loss: 0.6147
Epoch 5, training loss: 0.5677
Epoch 6, training loss: 0.4948
Epoch 7, training loss: 0.4114
Epoch 8, training loss: 0.3996
Epoch 9, training loss: 0.3288
Epoch 10, training loss: 0.2399
Epoch 11, training loss: 0.2265
Epoch 12, training loss: 0.1683
Epoch 13, training loss: 0.1567
Epoch 14, training loss: 0.1151
Epoch 15, training loss: 0.1041
Epoch 16, training loss: 0.1024
Epoch 17, training loss: 0

In [5]:
class MyModel(nn.Module):
    def __init__(self, original_model):
        super(MyModel, self).__init__()
        self.conv1 = original_model.conv1
        self.fc1 = original_model.fc1

    def forward(self, x):
        x = x.view(-1, 65, 256, 240)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = x.view(x.size(0), -1)  # Flatten layer
        x = F.relu(self.fc1(x))
        return x

# Create the new model
new_model = MyModel(model)

In [6]:
outputs = [] #stores one feature/value for each image for each patient.  Shape is (*,32,65).
labels_train = []
for data in train_dataloader:
    input_data = data[0].float()
    labels_train.append(data[1])
    output = new_model(input_data.to(device))
    outputs.append(output.detach().cpu().numpy()) 
flattened_outputs = np.concatenate(outputs, axis=0)
flattened_labels = np.concatenate(labels_train, axis = 0)
del outputs, output
np.save('modified_train.npy', flattened_outputs)
np.save('modified_labels.npy', flattened_labels)

In [7]:
del train_dataloader, train_dataset

In [8]:
X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')
test_dataset = MyDataset(X_test, y_test)

outputs_2 = []
# Assuming x_test is a PyTorch tensor
testloader = torch.utils.data.DataLoader(X_test, batch_size=32)
del X_test
for data in testloader:
    input_data = data.float()
    output = new_model(input_data.to(device))
    outputs_2.append(output.detach().cpu().numpy()) 


flattened_outputs_2 = np.concatenate(outputs_2, axis=0)
del outputs_2, output

In [9]:
np.save('modified_test.npy',flattened_outputs_2)

In [1]:
import numpy as np
x_train = np.load('modified_train.npy')
y_train = np.load('modified_labels.npy')
x_test = np.load('modified_test.npy')
y_test = np.load('y_test.npy')

In [37]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler

# Suppose 'data' is your 2D numpy array, with each row a datapoint
# And 'labels' is a 1D numpy array containing the class label for each datapoint
data = np.vstack((x_train,x_test))  # Your data
labels = np.hstack((y_train,y_test))  # Your labels

# Split the data into training and testing sets first
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.25, random_state=42)

scale_pos_weight = sum(labels_train == 0) / sum(labels_train == 1)

# Define the XGBoost model
model = XGBClassifier(
    max_depth=5,  # Maximum depth of the trees
    learning_rate=0.03,  # Learning rate (eta)
    n_estimators=100,  # Number of training rounds
    objective='binary:logistic',  # Objective function for binary classification
    scale_pos_weight = scale_pos_weight,
    random_state=42  # Random seed
)

# Train the model with early stopping
eval_set = [(data_test, labels_test)]  # Validation set for early stopping
model.fit(data_train, labels_train,eval_metric="logloss", eval_set=eval_set)

# Make predictions on the test set
preds_prob = model.predict_proba(data_test)

# 'preds_prob' now contains the predicted probabilities of the positive class for each datapoint in the test set


[0]	validation_0-logloss:0.67461
[1]	validation_0-logloss:0.65708
[2]	validation_0-logloss:0.64003
[3]	validation_0-logloss:0.62418
[4]	validation_0-logloss:0.60934
[5]	validation_0-logloss:0.59588
[6]	validation_0-logloss:0.58219
[7]	validation_0-logloss:0.56986
[8]	validation_0-logloss:0.55823
[9]	validation_0-logloss:0.54709
[10]	validation_0-logloss:0.53690
[11]	validation_0-logloss:0.52668
[12]	validation_0-logloss:0.51691
[13]	validation_0-logloss:0.50839
[14]	validation_0-logloss:0.49958
[15]	validation_0-logloss:0.49177
[16]	validation_0-logloss:0.48348
[17]	validation_0-logloss:0.47648
[18]	validation_0-logloss:0.46921
[19]	validation_0-logloss:0.46211
[20]	validation_0-logloss:0.45443
[21]	validation_0-logloss:0.44746
[22]	validation_0-logloss:0.44075
[23]	validation_0-logloss:0.43436
[24]	validation_0-logloss:0.42831
[25]	validation_0-logloss:0.42232
[26]	validation_0-logloss:0.41582
[27]	validation_0-logloss:0.41040
[28]	validation_0-logloss:0.40510
[29]	validation_0-loglos

C:\Users\thetu\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[32]	validation_0-logloss:0.38738
[33]	validation_0-logloss:0.38274
[34]	validation_0-logloss:0.37897
[35]	validation_0-logloss:0.37533
[36]	validation_0-logloss:0.37138
[37]	validation_0-logloss:0.36794
[38]	validation_0-logloss:0.36499
[39]	validation_0-logloss:0.36183
[40]	validation_0-logloss:0.35941
[41]	validation_0-logloss:0.35654
[42]	validation_0-logloss:0.35302
[43]	validation_0-logloss:0.35059
[44]	validation_0-logloss:0.34819
[45]	validation_0-logloss:0.34592
[46]	validation_0-logloss:0.34379
[47]	validation_0-logloss:0.34179
[48]	validation_0-logloss:0.33950
[49]	validation_0-logloss:0.33737
[50]	validation_0-logloss:0.33560
[51]	validation_0-logloss:0.33373
[52]	validation_0-logloss:0.33221
[53]	validation_0-logloss:0.33085
[54]	validation_0-logloss:0.32906
[55]	validation_0-logloss:0.32779
[56]	validation_0-logloss:0.32682
[57]	validation_0-logloss:0.32583
[58]	validation_0-logloss:0.32435
[59]	validation_0-logloss:0.32317
[60]	validation_0-logloss:0.32195
[61]	validatio

In [38]:
# Make predictions on the test set
def find_optimal_threshold(predictions, y_test):
    min_sum = float('inf')
    optimal_threshold = 0

    # Iterate over possible thresholds from 0 to 1
    for threshold in np.arange(0.0, 1, 0.001):
        # Apply threshold
        preds = (np.array(predictions)[:,1] > threshold).astype(int)

        # Compute confusion matrix
        cm = confusion_matrix(y_test, preds)

        # Compute sum of off-diagonal elements
        off_diagonal_sum = 164 - np.trace(cm)
        #print(cm)
        # Update optimal threshold if this threshold is better
        if off_diagonal_sum < min_sum and cm[1][1]/np.sum(cm[1]) >0.5:
            min_sum = off_diagonal_sum
            optimal_threshold = threshold

    return optimal_threshold

threshold = find_optimal_threshold(preds_prob, labels_test)
preds = (preds_prob[:, 1]> threshold).astype(int)

# Calculate the accuracy of the model
accuracy = accuracy_score(labels_test, preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 91.22%


In [39]:
from sklearn.metrics import roc_auc_score
roc_auc_score(labels_test, preds)

0.9042857142857144

In [40]:
from sklearn.metrics import confusion_matrix
confusion_matrix(labels_test, preds)

array([[91,  7],
       [ 6, 44]], dtype=int64)